# Mapping Melbourne with Leaflet

In this notebook we will be utilising the open-source mapping package Leaflet [https://leafletjs.com/](https://leafletjs.com/), and using it to plot data published by the Library on the Data Vic platform [https://www.data.vic.gov.au/](https://www.data.vic.gov.au/).

## Installing the Leaflet Python package

Leaflet is a JavaScript package, however there is a Jupyter widget version available that allows you to add maps to notebooks using Python [https://ipyleaflet.readthedocs.io/en/latest/](https://ipyleaflet.readthedocs.io/en/latest/).

Let's install the widget using the python package manager `pip`:

In [ ]:
!pip install ipyleaflet

## Getting the data

The Library has published a number of datasets to Data Vic that cabe be explored here: [https://discover.data.vic.gov.au//dataset/?organization=state-library-of-victoria](https://discover.data.vic.gov.au//dataset/?organization=state-library-of-victoria)

For this notebook we will use the Melbourne City Landmarks dataset - [https://github.com/statelibraryvic/opendata/blob/master/melbourne_city_landmarks.csv](https://github.com/statelibraryvic/opendata/blob/master/melbourne_city_landmarks.csv)

To read and load this data we will use a Python package called "Pandas" which is a shortened (and more fun) version of PANelled DAta. Pandas creates "dataframes" that are similar to spreadsheets, but can be easily manipulated and have various analyses run on them in the computers memory.

Pandas allows us to read the csv data directly from the 'raw' URL. It will create a DataFrame which we will store in a variable called `df`.

In [ ]:
import pandas as pd

github_url = "https://raw.githubusercontent.com/statelibraryvic/opendata/master/melbourne_city_landmarks.csv"

df = pd.read_csv(github_url)

df

,PID,Title,Description,Identifier,Format,Type,Access rights statement,Copyright statement,Relationship,Digital URI,ILMS Identifier,ILMS URI,Creator/Contributor,Date,Location,Accession number
0,2974587,"Alkira House. 18 Queen St, Melbourne","Alkira House , a 1937 six storey inter-war bui...",jc019534,image/jpg,StillImage,Use of this work allowed provided the creator ...,This work is in copyright. Copyright has been ...,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/2975890,1700142,http://search.slv.vic.gov.au/MAIN:Everything:S...,"Collins, John T. 1907-2001 , photographer.",1978-01-08,"""-37.818325, 144.962536"", 18 Queen",H98.252/1209
1,932957,Melbourne Post Office And Post Office Club Hotel,"The Melbourne General Post Office, otherwise k...",pi011466,image/jpg,StillImage,No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/933436,2100310,http://search.slv.vic.gov.au/MAIN:Everything:S...,"Kent, Bob, 1909?- compiler.",1946,"Cnr Bourke street and Elizabeth street, Melbou...",H2010.120/2
2,414255,"Looking West Collins St., Melbourne",The MLC (Mutual Life and Citizens Assurance) B...,rg004806,image/jpg,StillImage,No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/419506,1768560,http://search.slv.vic.gov.au/MAIN:Everything:S...,Rose Stereograph Co,1920/1954,"""-37.816611, 144.963815"", 303-309 Collins",H32492/4835
3,3079088,"St. Vincent's Hospital, Victoria Parade, Fitzroy","Part of Folding souvenir of Melbourne, 17 arti...",pi006829,image/jpg,"StillImage,Postcards",No copyright restrictions apply.,This work is out of copyright,Victorian state schools and students Series No. 4,http://api.slv.vic.gov.au/access_record/3080295,1807041,http://search.slv.vic.gov.au/MAIN:Everything:S...,NaN,1920/1929,"Victoria Parade, Fitzroy",H2008.12/120
4,1126156,Harry Rickards' Opera House & Prince Of Wales ...,The Tivoli Arcade is all that is left of the o...,a16925,image/jpg,StillImage,No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/1126354,1698470,http://search.slv.vic.gov.au/MAIN:Everything:S...,"Rudd, Charles 1849-1901 photographer.",1892/1900,"""-37.81334305, 144.9663239"", 249 Bourke",H39357/188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,3077646,"State Theatre, Flinders St. Melbourne","Once named the State Theatre, the Forum Theatr...",pc002856,image/jpg,"StillImage,Gelatin silver prints,Postcards",No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/3078388,1769418,http://search.slv.vic.gov.au/MAIN:Everything:S...,NaN,1933/1938,"""-37.81682587, 144.9693298"", 162 Flinders",H2000.222/19
330,3077650,Official Design For New Station At Spencer Str...,"Formerly Spencer Street Station, Southern Cros...",mp007357,image/jpg,StillImage,No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/3078390,1773780,http://search.slv.vic.gov.au/MAIN:Everything:S...,"Egersdorfer, Heiner.",1892-06-01,"""-37.81809235, 144.9538422"", 118 Spencer",IAN01/07/92/9
331,338188,Saint Pauls Cathedral Melbourne,Collection Of Postcard Views Of Melbourne Publ...,pc004489,image/jpg,StillImage,No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/338204,1913100,http://search.slv.vic.gov.au/MAIN:Everything:S...,"Cole, E. W. (Edward William), 1832-1918.",1902/1908,NaN,H2009.98/1
332,2039658,"Cnr Swanston and Collins Streets Melbourne, sh...",The Melbourne Town Hall was opened in 1870 and...,pc003329,image/jpg,"StillImage,Postcards",No copyright restrictions apply.,This work is out of copyright,Viewfolders Victoria. H-O.; Kookaburra series.,http://api.slv.vic.gov.au/access_record/2040146,1774350,http://search.slv.vic.gov.au/MAIN:Everything:S...,

## Inspecting our location data

Whenever you are working with data it is good practice to take a look at it and understand what you have.

Each landmark in the dataset has a Location. Here are the first 20 Locations:

In [ ]:
df["Location"].head(20)

,Location
0,"""-37.818325, 144.962536"", 18 Queen"
1,"Cnr Bourke street and Elizabeth street, Melbou..."
2,"""-37.816611, 144.963815"", 303-309 Collins"
3,"Victoria Parade, Fitzroy"
4,"""-37.81334305, 144.9663239"", 249 Bourke"
5,"""-37.816338, 144.952797"", 164 Spencer"
6,"361 Bourke Street, Melbourne, -37.81428528, 14..."
7,NaN
8,"""-37.816485, 144.966989"", Cnr Flinders & Swanston"
9,"""-37.818103, 144.965028"", Cnr Flinders & Eliza..."


In order to effectively create map plottings, Leaflet requires location data to be supplied as latitude and longitude pairings e.g. `(-37.8098,144.9652)`

The data in the Location column isn't formatted in this format. It tends to fall into one of three formats:

-  an address without geo coordinates e.g. Victoria Parade, Fitzroy
- an address that includes geo coordinates e.g. "-37.81334305, 144.9663239", 249 Bourke
- blank values i.e. NaN

### Scenario: an address without geo coordinate

We need to be able to convert a human readable address into a set of geo coordinates. Thankfully the `geopy` package provides the `Nominatim` class that can do just that:

In [ ]:
from geopy.geocoders import Nominatim

plain_text_location = "State Library Victoria, Melbourne"

app = Nominatim(user_agent="tutorial")

geo_location = app.geocode(plain_text_location)
geo_location.latitude, geo_location.longitude

(-37.809769599999996, 144.96553516995195)

### Scenario: an address that includes geo coordinates

We can use a "regular expression" to recognise a pattern we define, and extract the relevant data

**Note:** regular expressions can be difficult to write wtihout help. This is a useful tool for wrting and testing regualr expressions in Pyhton [https://pythex.org/](https://pythex.org/)

In [ ]:
import re

coord_text_location = '"-37.81334305, 144.9663239", 249 Bourke'

regex_pattern = re.compile("-*\d+\.\d+")
match = regex_pattern.findall(coord_text_location)

match

['-37.81334305', '144.9663239']

Next we can combine these two approaches into a function called `get_lat_long`, with some exception handling that means if no match is found, a list with two blank entries is returned.

In [ ]:
def get_lat_long(location: str):
    lat_long = []
    try:
        regex_pattern = re.compile("-*\d+\.\d+")
        match = regex_pattern.findall(location)
        if match:
            lat_long = match
        else:
            app = Nominatim(user_agent="tutorial")
            geo_location = app.geocode(location)
            lat_long =  [geo_location.latitude, geo_location.longitude]
        if len(lat_long) !=2:
          lat_long = ["", ""]
        return lat_long
    except Exception as e:
        return ["", ""]

Let's test our function against the three scenarios:

In [ ]:
print(get_lat_long(plain_text_location))
print(get_lat_long(coord_text_location))
print(get_lat_long("NaN"))

[-37.809769599999996, 144.96553516995195]
['-37.81334305', '144.9663239']
[46.3144754, 11.0480288]


Somewhat surprisingly, "NaN" returns a [location](https://maps.app.goo.gl/dijYZwXrF2fW4wVFA).

Therefore we should the pandas DataFrame method `notna()` to filter out the "NaN" or blank values and save that to  a new variable called `geo_df`

**Note** the `.shape` method will return the width (i.e. no. of columns) and and length of a dataframe.

In [ ]:
print("Before", df.shape)
geo_df = df[df["Location"].notna()]
print("After", geo_df.shape)

Before (334, 16)
After (239, 16)


Finally, we can use `apply()` on the DataFrame to call our function and add the geodata to two new columns: "latitude" and "longitude" before converting them to numeric values.

In [ ]:
# Call the get_lat_long funciton we created
geo_df[["latitude","longitude"]] = geo_df.apply(lambda x: get_lat_long(x["Location"]), axis="columns", result_type="expand")
# Convert the new columns to numeric values
geo_df[["latitude","longitude"]] = geo_df[["latitude","longitude"]].apply(pd.to_numeric, downcast="float", errors="coerce")
geo_df.head()

<ipython-input-36-7b9c63c8a4ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_df[["latitude","longitude"]] = geo_df.apply(lambda x: get_lat_long(x["Location"]), axis="columns", result_type="expand")
<ipython-input-36-7b9c63c8a4ad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_df[["latitude","longitude"]] = geo_df[["latitude","longitude"]].apply(pd.to_numeric, downcast="float", errors="coerce")


,PID,Title,Description,Identifier,Format,Type,Access rights statement,Copyright statement,Relationship,Digital URI,ILMS Identifier,ILMS URI,Creator/Contributor,Date,Location,Accession number,latitude,longitude
0,2974587,"Alkira House. 18 Queen St, Melbourne","Alkira House , a 1937 six storey inter-war bui...",jc019534,image/jpg,StillImage,Use of this work allowed provided the creator ...,This work is in copyright. Copyright has been ...,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/2975890,1700142,http://search.slv.vic.gov.au/MAIN:Everything:S...,"Collins, John T. 1907-2001 , photographer.",1978-01-08,"""-37.818325, 144.962536"", 18 Queen",H98.252/1209,-37.818325,144.962540
1,932957,Melbourne Post Office And Post Office Club Hotel,"The Melbourne General Post Office, otherwise k...",pi011466,image/jpg,StillImage,No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/933436,2100310,http://search.slv.vic.gov.au/MAIN:Everything:S...,"Kent, Bob, 1909?- compiler.",1946,"Cnr Bourke street and Elizabeth street, Melbou...",H2010.120/2,-37.814285,144.963303
2,414255,"Looking West Collins St., Melbourne",The MLC (Mutual Life and Citizens Assurance) B...,rg004806,image/jpg,StillImage,No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/419506,1768560,http://search.slv.vic.gov.au/MAIN:Everything:S...,Rose Stereograph Co,1920/1954,"""-37.816611, 144.963815"", 303-309 Collins",H32492/4835,-37.816612,144.963821
3,3079088,"St. Vincent's Hospital, Victoria Parade, Fitzroy","Part of Folding souvenir of Melbourne, 17 arti...",pi006829,image/jpg,"StillImage,Postcards",No copyright restrictions apply.,This work is out of copyright,Victorian state schools and students Series No. 4,http://api.slv.vic.gov.au/access_record/3080295,1807041,http://search.slv.vic.gov.au/MAIN:Everything:S...,NaN,1920/1929,"Victoria Parade, Fitzroy",H2008.12/120,-37.807739,144.973236
4,1126156,Harry Rickards' Opera House & Prince Of Wales ...,The Tivoli Arcade is all that is left of the o...,a16925,image/jpg,StillImage,No copyright restrictions apply.,This work is out of copyright,Hoddles Grid App,http://api.slv.vic.gov.au/access_record/1126354,1698470,http://search.slv.vic.gov.au/MAIN:Everything:S...,"Rudd, Charles 1849-1901 photographer.",1892/1900,"""-37.81334305, 144.9663239"", 249 Bourke",H39357/188,-37.813343,144.966324


## Mapping with iPyLeaflet

Now that we have our geocoordinate data we can begin to create some maps in Leaflet.

`iPyLeaflet` is the Jupyter notebook implementation of Leaflet.js, and it allows us to create interactive maps with a few lines of code.

To begin with, we'll create a map with Melbourne at it's center.

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

melbourne_coords = (-37.8136,144.9631)

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=melbourne_coords,
    zoom=16,
)

m

Map(center=[-37.8136, 144.9631], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

### iPyWidget Layout

Because iPyLeaflet is part of the `iPyWidgets` family, you can use the styling classes interoperably e.g. we can use the `Layout` class to change the size of the map.

In [ ]:
from ipywidgets import Layout

m.layout=Layout(width='80%', height="700px")

m

Map(bottom=2573888.0, center=[-37.81359819874736, 144.9630975723267], controls=(ZoomControl(options=['position…

### Layers

Layers can be used to add different map styles. The basemaps that are avaialbe are documented here [https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html](https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html)


A `LayersControl` allows the user to toggle between the different basemaps.



In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles
from ipyleaflet import LayersControl

openstreet_base_map = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
openstreet_base_map.base = True
positron_base_map = basemap_to_tiles(basemaps.CartoDB.Positron)
positron_base_map.base = True
strava_base_map = basemap_to_tiles(basemaps.Strava.All)
strava_base_map.base = True
nat_geo_map = basemap_to_tiles(basemaps.Esri.NatGeoWorldMap)
nat_geo_map.base = True

m = Map(
    layers=[positron_base_map, nat_geo_map, strava_base_map, openstreet_base_map],
    center=melbourne_coords,
    zoom=12,
    layout=Layout(width='80%', height="500px")
)

m.add(LayersControl())

m

Map(center=[-37.8136, 144.9631], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

### Markers

The `Marker` object is used to add markers to maps. Here we can add one using Melbourne's coordinates, with the default styling applied.

In [ ]:
from ipyleaflet import Marker

m.add(Marker(location=melbourne_coords))

m

Map(bottom=643672.0, center=[-37.8136, 144.9631], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

The `AwesomeIcon` object allows us to add Font Awesome web application icons. Here is the full list of compatible icons [https://fontawesome.com/v4/icons/](https://fontawesome.com/v4/icons/)

In the code block below, we will add a (spinning!) book icon to the map at the Library's coordinates.

The `Popup` object allows us to add some content that will display when the icon is clicked. We'll add some simple HTML content that will display a title and picture of the Library's La trobe reading room.

In [ ]:
from ipywidgets import HTML
from ipyleaflet import AwesomeIcon, Popup

slv_icon = AwesomeIcon(
    name="magic",
    icon_color='darkred',
    spin=True,
    icon_size=100
)

slv_marker = Marker(icon=slv_icon, location=(-37.8098,144.9652))

img_url = 'https://www.slv.vic.gov.au/sites/default/files/styles/feature_image/public/La%20Trobe%20Reading%20Room%20wide%202.JPG?itok=9i82tRMV'
img_html = f"<img src='{img_url}' width='100', height='70'>"
message = HTML()
message.value = f'<h3>State Library Victoria</h3> {img_html}'

popup = Popup(
        child=message,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )

slv_marker.popup = message

m.add(slv_marker)

m

Map(bottom=1287094.0, center=[-37.813581246980014, 144.96305465698245], controls=(ZoomControl(options=['positi…

### Plotting Melbourne's Landmarks

Now we can combine the landark data that we prepared earlier and stored in the `geo_df` dataframe with the map we've created.

To do this we will loop over the entries in the `geo_df` dataframe and add a `landmark_icon`, with popup containing the title and description from the original dataset, at the relevant coordinates.

In [ ]:
from ipyleaflet import AwesomeIcon, basemaps, Map, Marker, Popup
from ipywidgets import HTML, Layout

landmarks_map = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=melbourne_coords,
    zoom=14
)

landmarks_map.layout=Layout(width='80%', height="700px")


landmark_icon = AwesomeIcon(
    name="institution",
    icon_color='blue',
    marker_color="white",
    spin=True
)

for idx in geo_df.index:

    landmark_marker = Marker(icon=landmark_icon, location=(geo_df["latitude"][idx], geo_df["longitude"][idx]), title=geo_df["Title"][idx], opacity=0.5)
    landmarks_map.add(landmark_marker)

    message = HTML()
    message.value = f"""
                        <h3>{geo_df["Title"][idx]}</h3>
                        <p>{geo_df["Description"][idx]}</p>
                        <p><a href="https://find.slv.vic.gov.au/discovery/search?query=any,contains,{geo_df["Identifier"][idx]}&vid=61SLV_INST:SLV&search_scope=slv_local&tab=searchProfile" target="_blank">Link to catalogue</a></p>
                        """

    popup = Popup(
        child=message,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )

    landmark_marker.popup = message

landmarks_map

Map(center=[-37.8136, 144.9631], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…